In [1]:
# Importing necessary library
import pandas as pd
import numpy as np
import nltk
import os
import nltk.corpus
import re
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from numpy import array
from numpy import count_nonzero
from scipy.sparse import csr_matrix
import string
from scipy import sparse
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('treebank')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn import svm
from sklearn.svm import SVC
from sklearn import svm, datasets
from astropy.table import QTable, Table, Column
from matplotlib import pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'nltk'

In [2]:
df = pd.read_csv("Super_Hero_Dataset.csv", sep=",", header=None, engine='python', encoding="utf-8-sig") # imports the dataset
# Create a new variable called 'header' from the first row of the dataset
# This calls the first row for the header
header = df.iloc[0]
# take the rest of your data minus the header row
df = df[1:]
# set the header row as the df header
df.columns = header
df

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
1,3D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...","As 3-D Man, Chuck Chandler possessed approxima...",85,30,60,60,...,0,0,0,0,0,1,0,0,0,1
2,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,Enhanced Strength: 514A demonstrated great str...,100,20,30,50,...,0,0,0,0,1,0,1,0,0,1
3,A.I.M. Agent,NaN,NaN,7,A.I.M. agents are both soldiers and scientists...,NaN,90,5,20,25,...,0,0,1,0,0,0,0,0,0,0
4,A.M.A.Z.O. (CW),NaN,NaN,22,"The Anti Metahuman Adaptive Zootomic Organism,...","With its metallic body, it easily overpowered ...",90,100,100,100,...,1,0,1,0,0,0,0,0,0,0
5,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,Zane,Zane,Zane,25,Zane was created by Dr. Julien in a small work...,"As a Nindroid, Zane has increased stamina and ...",100,35,50,60,...,1,0,1,1,1,0,1,1,1,1
1359,Zatanna,Zatanna Zatara,Zatanna Zatara,10,Zatanna is the daughter of adventurer John Zat...,Zatanna is genetically talented with her magi...,90,10,25,30,...,0,0,0,0,0,0,0,0,0,0
1360,Zoom,Hunter Zolomon,Hunter Zolomon,9,Hunter Zolomon had a troubled relationship wi...,"Zoom is able to alter time, to make himself ev...",75,10,100,30,...,0,0,0,0,0,1,0,0,0,0
1361,Zoom (New 52),Hunter Zolomon,NaN,20,"Hunter Zolomon is better known as Zoom, a spee...",After tricking Barry Allen and Wally West into...,95,50,100,75,...,0,1,0,1,1,1,1,1,1,1


In [3]:
# changes all the ability variables to a numeric type from an object type. 
df['intelligence_score'] = pd.to_numeric(df['intelligence_score'], errors='coerce')
df['strength_score'] = pd.to_numeric(df['strength_score'], errors='coerce')
df['speed_score'] = pd.to_numeric(df['speed_score'], errors='coerce')
df['durability_score'] = pd.to_numeric(df['durability_score'], errors='coerce')
df['power_score'] = pd.to_numeric(df['power_score'], errors='coerce')
df['combat_score'] = pd.to_numeric(df['combat_score'], errors='coerce')

In [5]:
len(df)

1362

In [14]:
df.dtypes

0
name                  object
real_name             object
full_name             object
overall_score         object
history_text          object
                       ...  
has_super_speed       object
has_durability        object
has_stamina           object
has_agility           object
has_super_strength    object
Length: 81, dtype: object

In [7]:
df['attribute_score'] = df.apply(lambda x: x['intelligence_score'] + x['strength_score']+ x['speed_score']+ x['durability_score'] + x['power_score']+ x['combat_score'], axis=1)
df

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength,attribute_score
1,3D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...","As 3-D Man, Chuck Chandler possessed approxima...",85,30,60,60,...,0,0,0,0,1,0,0,0,1,345
2,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,Enhanced Strength: 514A demonstrated great str...,100,20,30,50,...,0,0,0,1,0,1,0,0,1,335
3,A.I.M. Agent,NaN,NaN,7,A.I.M. agents are both soldiers and scientists...,NaN,90,5,20,25,...,0,1,0,0,0,0,0,0,0,235
4,A.M.A.Z.O. (CW),NaN,NaN,22,"The Anti Metahuman Adaptive Zootomic Organism,...","With its metallic body, it easily overpowered ...",90,100,100,100,...,0,1,0,0,0,0,0,0,0,590
5,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,...,0,0,0,0,0,0,0,0,0,385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,Zane,Zane,Zane,25,Zane was created by Dr. Julien in a small work...,"As a Nindroid, Zane has increased stamina and ...",100,35,50,60,...,0,1,1,1,0,1,1,1,1,445
1359,Zatanna,Zatanna Zatara,Zatanna Zatara,10,Zatanna is the daughter of adventurer John Zat...,Zatanna is genetically talented with her magi...,90,10,25,30,...,0,0,0,0,0,0,0,0,0,310
1360,Zoom,Hunter Zolomon,Hunter Zolomon,9,Hunter Zolomon had a troubled relationship wi...,"Zoom is able to alter time, to make himself ev...",75,10,100,30,...,0,0,0,0,1,0,0,0,0,345
1361,Zoom (New 52),Hunter Zolomon,NaN,20,"Hunter Zolomon is better known as Zoom, a spee...",After tricking Barry Allen and Wally West into...,95,50,100,75,...,1,0,1,1,1,1,1,1,1,500


In [8]:
df.dropna(subset = ["history_text"], inplace=True)
len(df)

1278

In [9]:
df.dropna(subset = ["powers_text"], inplace=True)
len(df)

944

In [10]:
df.head(5)

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength,attribute_score
1,3D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...","As 3-D Man, Chuck Chandler possessed approxima...",85,30,60,60,...,0,0,0,0,1,0,0,0,1,345
2,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,Enhanced Strength: 514A demonstrated great str...,100,20,30,50,...,0,0,0,1,0,1,0,0,1,335
4,A.M.A.Z.O. (CW),NaN,NaN,22,"The Anti Metahuman Adaptive Zootomic Organism,...","With its metallic body, it easily overpowered ...",90,100,100,100,...,0,1,0,0,0,0,0,0,0,590
8,Abe Sapien,Abraham Sapien,Abraham Sapien,10,"Sapien began life as Langdon Everett Caul, a ...",Abe is a humanoid amphibious creature. He has...,95,30,35,65,...,1,1,1,1,0,1,1,1,1,410
9,Abin Sur,NaN,NaN,9,Originally a history professor on the planet ...,Abin Sur possessed an exceptionally strong s...,75,90,55,65,...,0,0,0,0,0,0,0,0,0,450


In [11]:
cols = df.select_dtypes(include=[np.object]).columns
df[cols] = df[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
df

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength,attribute_score
1,3D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...","As 3-D Man, Chuck Chandler possessed approxima...",85,30,60,60,...,0,0,0,0,1,0,0,0,1,345
2,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,Enhanced Strength: 514A demonstrated great str...,100,20,30,50,...,0,0,0,1,0,1,0,0,1,335
4,A.M.A.Z.O. (CW),NaN,NaN,22,"The Anti Metahuman Adaptive Zootomic Organism,...","With its metallic body, it easily overpowered ...",90,100,100,100,...,0,1,0,0,0,0,0,0,0,590
8,Abe Sapien,Abraham Sapien,Abraham Sapien,10,"Sapien began life as Langdon Everett Caul, a ...",Abe is a humanoid amphibious creature. He has...,95,30,35,65,...,1,1,1,1,0,1,1,1,1,410
9,Abin Sur,NaN,NaN,9,Originally a history professor on the planet ...,Abin Sur possessed an exceptionally strong s...,75,90,55,65,...,0,0,0,0,0,0,0,0,0,450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1357,Yukio (FOX),Yukio,Yukio,5,"When she was 5 years old, Yukio predicted her...","Yukio is a mutant with a limited, precognitiv...",80,10,50,30,...,0,1,0,1,0,0,1,1,0,315
1358,Zane,Zane,Zane,25,Zane was created by Dr. Julien in a small work...,"As a Nindroid, Zane has increased stamina and ...",100,35,50,60,...,0,1,1,1,0,1,1,1,1,445
1359,Zatanna,Zatanna Zatara,Zatanna Zatara,10,Zatanna is the daughter of adventurer John Zat...,Zatanna is genetically talented with her magi...,90,10,25,30,...,0,0,0,0,0,0,0,0,0,310
1360,Zoom,Hunter Zolomon,Hunter Zolomon,9,Hunter Zolomon had a troubled relationship wi...,"Zoom is able to alter time, to make himself ev...",75,10,100,30,...,0,0,0,0,1,0,0,0,0,345


In [9]:
for i, col in enumerate(df.columns):
    df.iloc[:, i] = df.iloc[:, i].str.replace('"', '')

In [10]:
df.head(5)

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
1,3D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...","As 3-D Man, Chuck Chandler possessed approxima...",85,30,60,60,...,0,0,0,0,0,1,0,0,0,1
2,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,Enhanced Strength: 514A demonstrated great str...,100,20,30,50,...,0,0,0,0,1,0,1,0,0,1
4,A.M.A.Z.O. (CW),NaN,NaN,22,"The Anti Metahuman Adaptive Zootomic Organism,...","With its metallic body, it easily overpowered ...",90,100,100,100,...,1,0,1,0,0,0,0,0,0,0
8,Abe Sapien,Abraham Sapien,Abraham Sapien,10,"Sapien began life as Langdon Everett Caul, a ...",Abe is a humanoid amphibious creature. He has...,95,30,35,65,...,0,1,1,1,1,0,1,1,1,1
9,Abin Sur,NaN,NaN,9,Originally a history professor on the planet ...,Abin Sur possessed an exceptionally strong s...,75,90,55,65,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df.dtypes

0
name                  object
real_name             object
full_name             object
overall_score         object
history_text          object
                       ...  
has_super_speed       object
has_durability        object
has_stamina           object
has_agility           object
has_super_strength    object
Length: 81, dtype: object

In [12]:
# importing word_tokenize from nltk
from nltk.tokenize import word_tokenize

In [13]:
df['token1'] = df.apply(lambda x: word_tokenize(x['powers_text']), axis=1)

def tokenize(value):
    if value is not NaN:
           return word_tokenizer(value)
    else:
        return []


In [14]:
df['token2'] = df.apply(lambda x: word_tokenize(x['history_text']), axis=1)

def tokenize(value):
    if value is not NaN:
           return word_tokenizer(value)
    else:
        return []

In [15]:
df['token1']

1       [As, 3-D, Man, ,, Chuck, Chandler, possessed, ...
2       [Enhanced, Strength, :, 514A, demonstrated, gr...
4       [With, its, metallic, body, ,, it, easily, ove...
8       [Abe, is, a, humanoid, amphibious, creature, ....
9       [Abin, Sur, possessed, an, exceptionally, stro...
                              ...                        
1357    [Yukio, is, a, mutant, with, a, limited, ,, pr...
1358    [As, a, Nindroid, ,, Zane, has, increased, sta...
1359    [Zatanna, is, genetically, talented, with, her...
1360    [Zoom, is, able, to, alter, time, ,, to, make,...
1361    [After, tricking, Barry, Allen, and, Wally, We...
Name: token1, Length: 944, dtype: object

In [16]:
df['token2']

1       [Delroy, Garrett, ,, Jr., grew, up, to, become...
2       [He, was, one, of, the, many, prisoners, of, I...
4       [The, Anti, Metahuman, Adaptive, Zootomic, Org...
8       [Sapien, began, life, as, Langdon, Everett, Ca...
9       [Originally, a, history, professor, on, the, p...
                              ...                        
1357    [When, she, was, 5, years, old, ,, Yukio, pred...
1358    [Zane, was, created, by, Dr., Julien, in, a, s...
1359    [Zatanna, is, the, daughter, of, adventurer, J...
1360    [Hunter, Zolomon, had, a, troubled, relationsh...
1361    [Hunter, Zolomon, is, better, known, as, Zoom,...
Name: token2, Length: 944, dtype: object

In [17]:
# # sample text for performing tokenization
text = df['powers_text']
# # use pandas series apply (kind of like the df apply)
text.apply(word_tokenize)
# # loop through the series
# for t in text:
#     words = word_tokenize(t)
#     print(words)

words1 = []

for t in text:
    t = t.lower()
    t = t.translate(str.maketrans('', '', string.punctuation))
    
    if t != '':
        words1+= word_tokenize(t)

#print(words1)

In [18]:
# # sample text for performing tokenization
text = df['history_text']
# # use pandas series apply (kind of like the df apply)
text.apply(word_tokenize)
# # loop through the series
# for t in text:
#     words = word_tokenize(t)
#     print(words)
words2 = []
for t in text:
    t = t.lower()
    t = t.translate(str.maketrans('', '', string.punctuation))
    
    if t != '':
        words2 += word_tokenize(t)

#print(words2)

In [19]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [20]:
stop_words = set(stopwords.words('english'))

filtered_sentence1 = [w for w in words1 if not w in stop_words] 
  
filtered_sentence1 = [] 
  
for w in words1: 
    if w not in stop_words: 
        filtered_sentence1.append(w)

#print(filtered_sentence1) 

In [21]:
stop_words = set(stopwords.words('english'))

filtered_sentence2 = [w for w in words2 if not w in stop_words] 
  
filtered_sentence2 = [] 
  
for w in words2: 
    if w not in stop_words: 
        filtered_sentence2.append(w) 
  
 
print(filtered_sentence2) 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
# finding the frequency distinct in the tokens
# Importing FreqDist library from nltk and passing token into FreqDist
from nltk.probability import FreqDist
fdist1 = FreqDist(filtered_sentence1)
fdist1

FreqDist({'able': 1561, 'also': 1371, 'even': 953, 'ability': 829, 'energy': 767, 'powers': 738, 'strength': 735, 'body': 725, 'power': 722, 'physical': 633, ...})

In [23]:
from nltk.probability import FreqDist
fdist2 = FreqDist(filtered_sentence2)
fdist2

FreqDist({'new': 1563, 'time': 1482, 'later': 1377, 'would': 1343, 'one': 1338, 'however': 1125, 'team': 1019, 'power': 945, 'powers': 937, 'earth': 927, ...})

In [24]:
# To find the frequency of top 5 words
fdist1_top50 = fdist1.most_common(500)
fdist1_top50

[('able', 1561),
 ('also', 1371),
 ('even', 953),
 ('ability', 829),
 ('energy', 767),
 ('powers', 738),
 ('strength', 735),
 ('body', 725),
 ('power', 722),
 ('physical', 633),
 ('human', 625),
 ('time', 577),
 ('force', 556),
 ('superhuman', 544),
 ('speed', 511),
 ('use', 483),
 ('one', 476),
 ('form', 469),
 ('could', 442),
 ('capable', 440),
 ('abilities', 433),
 ('without', 427),
 ('enough', 397),
 ('however', 390),
 ('control', 368),
 ('powerful', 356),
 ('possesses', 354),
 ('like', 353),
 ('well', 353),
 ('shown', 345),
 ('great', 338),
 ('used', 335),
 ('hulk', 330),
 ('skilled', 326),
 ('using', 318),
 ('combat', 302),
 ('due', 276),
 ('normal', 273),
 ('several', 266),
 ('much', 261),
 ('enhanced', 258),
 ('would', 254),
 ('though', 247),
 ('black', 242),
 ('create', 239),
 ('level', 237),
 ('objects', 227),
 ('fight', 225),
 ('despite', 217),
 ('easily', 214),
 ('mind', 211),
 ('hand', 210),
 ('many', 210),
 ('people', 209),
 ('others', 209),
 ('allowing', 207),
 ('highly'

In [25]:
fdist2_top50 = fdist2.most_common(5)
fdist2_top50

[('new', 1563),
 ('time', 1482),
 ('later', 1377),
 ('would', 1343),
 ('one', 1338)]

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
# list of text documents
# create the transform
powers_vectorizer = TfidfVectorizer()
# tokenize and build vocab
powers_vectorizer.fit(filtered_sentence1)
# summarize
print(powers_vectorizer.idf_)
print(powers_vectorizer.vocabulary_)

[12.15679323 11.2405025  12.15679323 ... 11.75132812 12.15679323
 12.15679323]
{'3d': 144, 'man': 8438, 'chuck': 2490, 'chandler': 2358, 'possessed': 10485, 'approximately': 898, 'three': 14105, 'times': 14165, 'physical': 10251, 'capabilities': 2170, 'extremely': 5051, 'physically': 10253, 'fit': 5367, 'otherwise': 9832, 'normal': 9482, 'human': 6647, 'male': 8421, 'name': 9228, 'suggests': 13489, 'strong': 13365, 'fast': 5155, 'durable': 4305, 'military': 8867, 'pilot': 10291, 'sensory': 12280, 'acuity': 363, 'five': 5374, 'senses': 12272, 'powerful': 10523, 'maximum': 8617, 'addition': 390, 'limited': 8140, 'quasitelepathic': 10957, 'ability': 231, 'perceive': 10140, 'distinctive': 4065, 'aura': 1194, 'skrull': 12685, 'race': 10994, 'even': 4827, 'assumed': 1086, 'another': 790, 'form': 5546, 'expert': 4974, 'talented': 13780, 'football': 5496, 'player': 10377, 'hal': 6168, 'experienced': 4964, 'scientific': 12097, 'researcher': 11531, 'astigmatic': 1093, 'requires': 11523, 'special

In [32]:
# list of text documents
# create the transform
history_vectorizer = TfidfVectorizer()
# tokenize and build vocab
history_vectorizer.fit(filtered_sentence2)
# summarize
print(history_vectorizer.idf_)
print(history_vectorizer.vocabulary_)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [33]:
# encode document
powers = powers_vectorizer.transform(filtered_sentence1)
# summarize encoded vector
print('The Vectors Shape is ', powers.shape)
A=powers.toarray()
print(powers.toarray())

The Vectors Shape is  (140075, 15663)


MemoryError: Unable to allocate 16.3 GiB for an array with shape (140075, 15663) and data type float64

In [29]:
from scipy.stats import skew

x = A

print (skew(x))

NameError: name 'A' is not defined

In [ ]:
history = history_vectorizer.transform(filtered_sentence2)
# summarize encoded vector
print('The Vectors Shape is ', history.shape)
C=history.toarray()
print(history.toarray())

In [ ]:
V = set(filtered_sentence1)
long_words = [w for w in V if len(w) > 12]
sorted(long_words)

In [ ]:
for xyz in filtered_sentence1:
     if xyz.endswith('ness'):
         print(xyz)

In [ ]:
from collections import Counter


c = Counter(words1)
unique = [w for w in words1 if c[w] <= 30]

#print("Unique words: ", unique)
from nltk.probability import FreqDist
fdist3 = FreqDist(unique)
fdist3

In [ ]:
fdist3_top50 = fdist3.most_common(1000)
fdist3_top50

In [ ]:
# from sklearn.feature_extraction.text import HashingVectorizer
# # list of text documents
# # create the transform
# vectorizer = HashingVectorizer(n_features=2**12, binary=bool)
# # encode document
# A = vectorizer.fit_transform(words)
# # summarize encoded vector
# print(A.shape)
# print(A.toarray())
# # The values of the encoded document correspond to normalized word counts by default in the range of -1 to 1
# # the below array was onstructed from the word created dictionary via the powers text variable 

In [ ]:
# # conversts the array to a sparse metrix 
# S = csr_matrix(A)
# print(S.shape)
# print(S)

In [ ]:
# reconstructs the sparse matrix to a dense array
# csr_matrix.todense(S, order='C', out=None)

In [ ]:
w = csr_matrix.todense(powers, order=None, out=None)
# calculate sparsity
from numpy import array
from numpy import count_nonzero
# create dense matrix
sparsity = 1.0 - count_nonzero(w) / w.size
round(sparsity,4)

In [ ]:
w = csr_matrix.todense(history, order=None, out=None)
# calculate sparsity
from numpy import array
from numpy import count_nonzero
# create dense matrix
sparsity = 1.0 - count_nonzero(w) / w.size
round(sparsity,4)

In [ ]:
tagged = nltk.pos_tag(filtered_sentence1)
tagged[0:20]

In [ ]:
from collections import Counter
Counter(" ".join(df["history_text"]).split()).most_common(1000)

In [ ]:
Counter(" ".join(df["powers_text"]).split()).most_common(500)